In [1]:
from left_turn_pedestrian.controller import sys_ctrl
import pickle
import hard_tests
import random
import visualization.animate as animate

import tulip as tlp
from tulip import transys, abstract, spec, synth
from IPython.display import HTML

import networkx as nx

/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/mathset.py:719: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/mathset.py:719: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/labeled_graphs.py:1218: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/labeled_graphs.py:1226: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/python3.8/site-packages/tulip-1.4.0-py3.8.egg/tulip/transys/labeled_graphs.py:1218: SyntaxWarning: "is" with a literal. Did you mean "=="?
/Users/kimiahassibi/Desktop/Caltech/SURF2023/env/lib/pyt

In [2]:
path = 'left_turn_pedestrian/'

# Organizing the Graph

In [3]:
# Load the graph from the saved file
with open(path + 'graph', "rb") as file:
    G = pickle.load(file)

In [4]:
# Load the system controller from the saved file
with open(path + "ctrl", "rb") as file:
    ctrl = pickle.load(file)

In [5]:
# Labeling the nodes like system nodes
for edge in ctrl.edges:
    for (key, val) in ctrl.edges[edge].items():
        ctrl.nodes[edge[1]][key] = val

In [6]:
# Conversions
light_conversion = ["g1", "g2", "g3", "y1", "y2", "r"] # TODO: Double check this conversion
loc_conversion = ['c4', 'c7', 'c8', 'c9']
for node in G.nodes:
    G.nodes[node]['loc'] = loc_conversion[G.nodes[node]['loc']]
    G.nodes[node]['light'] = light_conversion[G.nodes[node]['light']]

In [7]:
len(G.nodes)

834

In [8]:
# Removing nodes that violate the environment's safety
unsafe_env_nodes = set()
for node in G.nodes:
    if G.nodes[node]['light'] == 'r':
        if (G.nodes[node]['p'] == 4 or G.nodes[node]['vh'] == 4 
        or G.nodes[node]['p'] == 5 or G.nodes[node]['vh'] == 5):
            unsafe_env_nodes.add(node)
    if G.nodes[node]['light'] == 'y2':
        if G.nodes[node]['p'] == 4 or G.nodes[node]['vh'] == 4:
            unsafe_env_nodes.add(node)
G.remove_nodes_from(unsafe_env_nodes)

In [9]:
len(G.nodes)

658

In [10]:
# Relabeling the nodes in order
nodes = list(G.nodes)
new_labels = list(range(len(nodes)))
mapping = dict(zip(nodes, new_labels))
G = nx.relabel_nodes(G, mapping)

# Creating the Potential Hard Tests

## Memoryless Most Red

In [11]:
num_red_sys_metric = hard_tests.find_num_red_successors(G)

In [12]:
memoryless_most_red = hard_tests.memoryless_max_metric(G, num_red_sys_metric)

# Memoryless Percent Red

In [13]:
percent_red_sys_metric = hard_tests.find_percent_red_successors(G)

In [14]:
memoryless_percent_red = hard_tests.memoryless_max_metric(G, percent_red_sys_metric)

## Memoryless Min Robustness - Minimizing 

In [15]:
min_robustness_env_metric = hard_tests.find_min_robustness(G)

In [16]:
memoryless_min_rob_minimizing = hard_tests.memoryless_robustness_minimization(G, min_robustness_env_metric)

## Memoryless Average Robustness - Minimizing

In [17]:
avg_robustness_env_metric = hard_tests.find_avg_robustness(G)

In [18]:
memoryless_avg_rob_minimizing = hard_tests.memoryless_robustness_minimization(G, avg_robustness_env_metric)

## Memoryless Min Robustness - Averaging

In [19]:
min_robustness_env_metric = hard_tests.find_min_robustness(G)

In [20]:
memoryless_min_rob_averaging = hard_tests.memoryless_robustness_averaging(G, min_robustness_env_metric)

## Memoryless Average Robustness - Averaging

In [21]:
avg_robustness_env_metric = hard_tests.find_avg_robustness(G)

In [22]:
memoryless_avg_rob_averaging = hard_tests.memoryless_robustness_averaging(G, avg_robustness_env_metric)

# Comparing Tests

In [23]:
memoryless_most_red == memoryless_percent_red

True

In [24]:
memoryless_avg_rob_minimizing == memoryless_min_rob_minimizing

True

In [25]:
memoryless_avg_rob_averaging == memoryless_min_rob_averaging

True

In [26]:
memoryless_min_rob_minimizing == memoryless_min_rob_averaging

True

In [27]:
memoryless_avg_rob_minimizing == memoryless_avg_rob_averaging

True

So all of the robustness tests are the same

In [28]:
memoryless_most_red == memoryless_avg_rob_minimizing

False

But the counting neighbors tests are not the same as the memoryless tests